In [110]:
import sqlite3
import pandas as pd
from datetime import datetime 

# Create the connection
db = sqlite3.connect('sqlite_db_pythonsqlite.db')

# create the dataframe from a query
bookings = pd.read_sql_query("SELECT * FROM bookings", db)
facilities = pd.read_sql_query("SELECT * FROM facilities", db)
members = pd.read_sql_query("SELECT * FROM members", db)

In [111]:
bookings.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [112]:
facilities

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


In [113]:
members.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue.

In [114]:
def total_revenue(facid):
    '''Calculates the total revenue of a given facility'''
    total_revenue = 0;
    member_cost = facilities.membercost[facilities.facid==facid][facid]
    guest_cost = facilities.guestcost[facilities.facid==facid][facid]
    fac_bookings = bookings[bookings.facid==facid]
    
    for i, row in fac_bookings.iterrows():
        if row.memid > 0: total_revenue += member_cost
        else : total_revenue += guest_cost
           
    return total_revenue

output = pd.DataFrame(columns=['Facility name', 'Total revenue'])
for i, row in facilities.iterrows():
    output.loc[i] = [facilities.name[i], total_revenue(i)]
    
output[output['Total revenue'] < 1000]

,Facility name,Total revenue
2,Badminton Court,604.5
3,Table Tennis,90.0
7,Snooker Table,115.0
8,Pool Table,265.0


Produce a report of members and who recommended them in alphabetic surname, firstname order.

In [115]:
member_names = pd.DataFrame(columns=['Member name', 'Recommender name'])
for i, row in members.sort_values('surname').iterrows():
    if i == 0: continue
    if members.recommendedby[i] == '': recommender = None
    else: recommender = (members.surname[int(members.recommendedby[i])] + ', ' + members.firstname[int(members.recommendedby[i])])
    member_names.loc[i] = [(members.surname[i] + ' ' + members.firstname[i]), recommender]
    
member_names

,Member name,Recommender name
15,Bader Florence,"Stibbons, Ponder"
16,Baker Timothy,"Farrell, Jemima"
12,Baker Anne,"Stibbons, Ponder"
8,Boothe Tim,"Rownam, Tim"
5,Butters Gerald,"Smith, Darren"
20,Coplin Joan,"Baker, Timothy"
29,Crumpet Erica,"Smith, Tracy"
7,Dare Nancy,"Joplette, Janice"
24,Farrell David,None
13,Farrell Jemima,None


Find the facilities with their usage by member, but not guests

In [142]:
fac_usage_mem = bookings[bookings.memid > 0].groupby(['facid', 'memid'])[['bookid']].count()
fac_usage_mem.index = fac_usage_mem.index.set_levels(list(facilities.name), level=0)
fac_usage_mem

bookid
facid          memid        
Tennis Court 1 2          30
               3           6
               4          19
               5          57
               6          31
...                      ...
Pool Table     27          3
               28         25
               29         33
               30          5
               33          8

[202 rows x 1 columns]

Find the facilities with their usage by month, but not guests

In [144]:
print('First Booking: ', min(bookings.starttime).split()[0],
     '\tLast Booking: ', max(bookings.starttime).split()[0])
bookings['month'] = bookings.starttime.apply(lambda x: x.split()[0].split('-')[1])
fac_usage_mon = bookings[bookings.memid > 0].groupby(['facid', 'month'])[['bookid']].count()
fac_usage_mon

First Booking:  2012-07-03 	Last Booking:  2012-09-30


bookid
facid month        
0     07         65
      08        111
      09        132
1     07         41
      08        109
      09        126
2     07         51
      08        132
      09        161
3     07         48
      08        143
      09        194
4     07         77
      08        153
      09        191
5     07          4
      08          9
      09         14
6     07         23
      08         85
      09         87
7     07         68
      08        154
      09        199
8     07        103
      08        272
      09        408